In [1]:
import gurobipy as gp
import pandas as pd

# read excel sheets
data = pd.ExcelFile('SuperChipData.xlsx')
productionCapacityDf = pd.read_excel(data, 'Production Capacity')
salesRegionDemandDf = pd.read_excel(data, 'Sales Region Demand')
shippingCostsDf = pd.read_excel(data, 'Shipping Costs')
ProductionCostsDf = pd.read_excel(data, 'Production Costs')
print(salesRegionDemandDf.to_numpy())

# facility S
facilitySet = {"Alexandria", "Richmond", "Norfolk", "Roanoke", "Charlottesville"}
# sales region j
salesRegion = 23
# computer chip z
computerChip = 30
# production capacity C
capacity = [348, 312, 294, 216, 204]
# sales region demand D
demand = []
        
        

m = gp.Model()
m.modelSense = gp.GRB.MINIMIZE

[[ 1.    1.    1.18]
 [ 1.    2.    2.17]
 [ 1.    3.    2.82]
 ...
 [23.   28.    2.55]
 [23.   29.    1.57]
 [23.   30.    2.87]]
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-25
